[Relationships](https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html) \
[QuickStart](https://docs.sqlalchemy.org/en/20/orm/quickstart.html)

In [32]:
import os.path as path
import os

from sqlalchemy import Column, Float, ForeignKey, Integer, String, Table, select, event
from sqlalchemy.engine import Engine, create_engine
from sqlalchemy.ext.associationproxy import association_proxy
from sqlalchemy.orm import DeclarativeBase, Session, relationship

In [33]:
@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

In [34]:
class Base(DeclarativeBase):
    pass


cashflows_tags_table = Table(
    "cashflows_tags",
    Base.metadata,
    Column("cashflow_id", ForeignKey("cashflows.id"), primary_key=True),
    Column("tag_id", ForeignKey("tags.id"), primary_key=True),
)


class Tag(Base):
    __tablename__ = "tags"

    id = Column(Integer, primary_key=True)
    tag = Column(String, nullable=False, unique=True)

    cash = relationship(
        "Cashflow", secondary=cashflows_tags_table, back_populates="tags"
    )


class Cashflow(Base):
    __tablename__ = "cashflows"

    id = Column(Integer, primary_key=True)
    amount = Column(Float, nullable=False)
    description = Column(String, nullable=False)

    # Many to Many
    tags = relationship("Tag", secondary=cashflows_tags_table, back_populates="cash")
    templates = association_proxy("tags", "amount")

In [35]:
engine = create_engine(f"sqlite://", echo=True)

# db = path.join(path.abspath(""), "test.db")
# if path.exists(db):
#     os.remove(db)
# engine = create_engine(f"sqlite:///{db}", echo=True)

Base.metadata.create_all(engine)

2024-02-21 22:13:51,680 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 22:13:51,681 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cashflows_tags")
2024-02-21 22:13:51,681 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-21 22:13:51,682 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cashflows_tags")
2024-02-21 22:13:51,683 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-21 22:13:51,684 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tags")
2024-02-21 22:13:51,684 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-21 22:13:51,684 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("tags")
2024-02-21 22:13:51,685 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-21 22:13:51,685 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cashflows")
2024-02-21 22:13:51,686 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-21 22:13:51,686 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cashflows")
2024-02-21 22:13:51,687 INFO sqlalchemy.engine.Engine 

In [36]:
with Session(engine) as session:

    tags = [Tag(id=1, tag="Tag1"), Tag(id=2, tag="Tag2"), Tag(id=3, tag="Tag3")]

    cash = [
        Cashflow(id=1, amount=100, description="cash1"),
        Cashflow(id=2, amount=100, description="cash1"),
        Cashflow(id=3, amount=100, description="cash1"),
    ]

    for t, c in zip(tags, cash):
        c.tags.append(t)

    session.add_all(tags)
    session.add_all(cash)
    session.commit()
    
    for c in cash:
        print(f"Amount: {c.amount}")

2024-02-21 22:13:51,909 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 22:13:51,911 INFO sqlalchemy.engine.Engine INSERT INTO cashflows (id, amount, description) VALUES (?, ?, ?)
2024-02-21 22:13:51,911 INFO sqlalchemy.engine.Engine [generated in 0.00077s] [(1, 100.0, 'cash1'), (2, 100.0, 'cash1'), (3, 100.0, 'cash1')]
2024-02-21 22:13:51,914 INFO sqlalchemy.engine.Engine INSERT INTO tags (id, tag) VALUES (?, ?)
2024-02-21 22:13:51,914 INFO sqlalchemy.engine.Engine [generated in 0.00058s] [(1, 'Tag1'), (2, 'Tag2'), (3, 'Tag3')]
2024-02-21 22:13:51,915 INFO sqlalchemy.engine.Engine INSERT INTO cashflows_tags (cashflow_id, tag_id) VALUES (?, ?)
2024-02-21 22:13:51,915 INFO sqlalchemy.engine.Engine [generated in 0.00037s] [(2, 2), (3, 3), (1, 1)]
2024-02-21 22:13:51,916 INFO sqlalchemy.engine.Engine COMMIT
2024-02-21 22:13:51,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 22:13:51,976 INFO sqlalchemy.engine.Engine SELECT cashflows.id AS cashflows_id, cashflows.am

In [37]:
with Session(engine) as session:
    tag = session.get(Tag, 1)

    session.delete(tag)
    session.commit()

2024-02-21 22:13:51,986 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 22:13:51,987 INFO sqlalchemy.engine.Engine SELECT tags.id AS tags_id, tags.tag AS tags_tag 
FROM tags 
WHERE tags.id = ?
2024-02-21 22:13:51,988 INFO sqlalchemy.engine.Engine [generated in 0.00069s] (1,)
2024-02-21 22:13:51,990 INFO sqlalchemy.engine.Engine SELECT cashflows.id AS cashflows_id, cashflows.amount AS cashflows_amount, cashflows.description AS cashflows_description 
FROM cashflows, cashflows_tags 
WHERE ? = cashflows_tags.tag_id AND cashflows.id = cashflows_tags.cashflow_id
2024-02-21 22:13:51,990 INFO sqlalchemy.engine.Engine [generated in 0.00045s] (1,)
2024-02-21 22:13:51,992 INFO sqlalchemy.engine.Engine DELETE FROM cashflows_tags WHERE cashflows_tags.cashflow_id = ? AND cashflows_tags.tag_id = ?
2024-02-21 22:13:51,992 INFO sqlalchemy.engine.Engine [generated in 0.00037s] (1, 1)
2024-02-21 22:13:51,994 INFO sqlalchemy.engine.Engine DELETE FROM tags WHERE tags.id = ?
2024-02-21 22:13:51,99

In [38]:
with Session(engine) as session:
    cash = session.get(Cashflow, 3)
    session.delete(cash)
    session.commit()

2024-02-21 22:13:52,047 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 22:13:52,048 INFO sqlalchemy.engine.Engine SELECT cashflows.id AS cashflows_id, cashflows.amount AS cashflows_amount, cashflows.description AS cashflows_description 
FROM cashflows 
WHERE cashflows.id = ?
2024-02-21 22:13:52,049 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (3,)
2024-02-21 22:13:52,051 INFO sqlalchemy.engine.Engine SELECT tags.id AS tags_id, tags.tag AS tags_tag 
FROM tags, cashflows_tags 
WHERE ? = cashflows_tags.cashflow_id AND tags.id = cashflows_tags.tag_id
2024-02-21 22:13:52,051 INFO sqlalchemy.engine.Engine [generated in 0.00093s] (3,)
2024-02-21 22:13:52,052 INFO sqlalchemy.engine.Engine DELETE FROM cashflows_tags WHERE cashflows_tags.cashflow_id = ? AND cashflows_tags.tag_id = ?
2024-02-21 22:13:52,053 INFO sqlalchemy.engine.Engine [cached since 0.06138s ago] (3, 3)
2024-02-21 22:13:52,055 INFO sqlalchemy.engine.Engine DELETE FROM cashflows WHERE cashflows.id = ?
2024-02-

In [39]:
with Session(engine) as session:
    cash = session.get(Cashflow, 1)
    tag = session.get(Tag, 2)

    print(cash.templates)
    print(cash.tags)
    print(tag.cash)

2024-02-21 22:13:52,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 22:13:52,131 INFO sqlalchemy.engine.Engine SELECT cashflows.id AS cashflows_id, cashflows.amount AS cashflows_amount, cashflows.description AS cashflows_description 
FROM cashflows 
WHERE cashflows.id = ?
2024-02-21 22:13:52,131 INFO sqlalchemy.engine.Engine [cached since 0.08355s ago] (1,)
2024-02-21 22:13:52,132 INFO sqlalchemy.engine.Engine SELECT tags.id AS tags_id, tags.tag AS tags_tag 
FROM tags 
WHERE tags.id = ?
2024-02-21 22:13:52,133 INFO sqlalchemy.engine.Engine [cached since 0.146s ago] (2,)
2024-02-21 22:13:52,134 INFO sqlalchemy.engine.Engine SELECT tags.id AS tags_id, tags.tag AS tags_tag 
FROM tags, cashflows_tags 
WHERE ? = cashflows_tags.cashflow_id AND tags.id = cashflows_tags.tag_id
2024-02-21 22:13:52,134 INFO sqlalchemy.engine.Engine [cached since 0.08439s ago] (1,)
[]
[]
2024-02-21 22:13:52,136 INFO sqlalchemy.engine.Engine SELECT cashflows.id AS cashflows_id, cashflows.amount AS ca